In [17]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import numpy as np
import pandas as pd

##### 울산 업소명 및 주소 추출하기

In [18]:
Ulsan = pd.read_csv('mini_project_1/Ulsan.csv')
Ulsan

,Unnamed: 0,업종,업소명,주요품목,가격,연락처,주소
0,0,한식_일반,경북식당,정식,7000원,052-267-7855,울산광역시 남구 돋질로 145번길 38-25
1,1,한식_일반,제주식당,된장찌개,7000원,052-232-2238,울산광역시 동구 명덕3길 14(서부동)
2,2,기타양식,하이밀베이커리,소보루빵,1700원,052-286-9900,"울산광역시 북구 아진로 94, 113, 114, 117호"
3,3,이미용업,세븐헤어샵,커트(성인),12000원,052-286-7665,울산광역시 북구 아진로 106-2
4,4,한식_일반,금낭상회,칼국수,6000원,0507-1444-0622,"울산광역시 울주군 온산읍 신온7길 57, 나동 112호"
...,...,...,...,...,...,...,...
112,112,이미용업,선경이용원,컷트,5000원,052-246-1703,울산광역시 중구 유곡로 10(우정동)
113,113,이미용업,현대이용원,커트,10000원,052-260-6123,울산광역시 남구 돋질로251번길 38
114,114,이미용업,경주이용원,컷트,12000원,010-4907-9990,울산광역시 남구 월평로159번길 4
115,115,중식,중화루,자장면,6000원,052-224-0392,울산광역시 남구 삼호로11번길 32


In [19]:
Ulsan['주소_구,군'] = Ulsan['주소'].apply(lambda x:x.split(' ')[0])

Ulsan['주소_구,군'] = Ulsan['주소'].apply(lambda x:x.replace('\xa0',' '))
Ulsan['keyword'] = Ulsan['주소_구,군']

Ulsan_address_list = Ulsan['keyword'].apply(lambda x:x.split(' '))

new_Ulsan_address_list = []

for i,item in enumerate(Ulsan_address_list):
    #print(ad_li[i][:2])
    address = item[0] + ' ' + item[1]
    new_Ulsan_address_list.append(address)
    
new_Ulsan_address_list

Ulsan['keyword'] = new_Ulsan_address_list

Ulsan['keyword'] = Ulsan['keyword'] + ' ' + Ulsan['업소명']

In [20]:
# 카카오맵 url 가져오기
url = 'https://map.kakao.com/'

# 크롬 드라이버 실행하기
driver = webdriver.Chrome()

# 카카오맵 url로 이동하기
driver.get(url)

score_list = []

for key in Ulsan['keyword'][:117]:
    # 검색창 클릭하기
    search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')

    # keyword의 key를 검색창에 넣기
    search.send_keys(key)

    # 엔터키로 이동하기
    search.send_keys(Keys.RETURN)

    time.sleep(2)

    # 웹 스크립트 받아오기
    html = driver.page_source

    # 웹스크립트에서 원하는 콘텐츠 추출하기 - parsing
    soup = BeautifulSoup(html, 'html.parser')
    
    # keyword별 평점 가져오기
    try:
        # 평점 path 가져오기
        score_path = '#info\.search\.place\.list > li:nth-child(1) > div.rating.clickArea'
        
        # score_path를 split한 후 3번째 문자 추출하기
        score = soup.select(score_path)[0].text.split('\n')[3]
        
        # 평점을 빈 리스트 L에 추가하기
        score_list.append(score)
        
    except:
        score_list.append('별점 없음')
        if soup.select(score_path)[0].text.split('\n')[9] == '후기 미제공':
            score_list.append('별점 없음')
        
    search.clear()
    
score_list

Ulsan['평점'] = score_list
Ulsan.to_csv('Ulsanscore.csv', encoding='utf-8-sig')

<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:33: SyntaxWarning: invalid escape sequence '\.'
<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:33: SyntaxWarning: invalid escape sequence '\.'
C:\Users\vvv\AppData\Local\Temp\ipykernel_12032\380626318.py:14: SyntaxWarning: invalid escape sequence '\.'
  search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')
C:\Users\vvv\AppData\Local\Temp\ipykernel_12032\380626318.py:33: SyntaxWarning: invalid escape sequence '\.'
  score_path = '#info\.search\.place\.list > li:nth-child(1) > div.rating.clickArea'
